## Running seisflows following 2D Example Walkthrough
https://seisflows.readthedocs.io/en/devel/2D_example_walkthrough.html   
by AR  

In [1]:
import os
import glob
import shutil
import numpy as np

In [2]:
# vvv USER MUST EDIT THE FOLLOWING PATHS vvv
WORKDIR = os.getcwd()
SPECFEM2D = "/home/scoped"
# where WORKDIR: points to your own working directory
# and SPECFEM2D: points to an existing specfem2D repository if available (if not set as '')
# ^^^ USER MUST EDIT THE FOLLOWING PATHS ^^^
# ======================================================================================================

# Distribute the necessary file structure of the SPECFEM2D repository that we will downloaded/reference
SPECFEM2D_ORIGINAL = os.path.join(SPECFEM2D, "specfem2d") 
SPECFEM2D_BIN_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "bin")
SPECFEM2D_DATA_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "DATA")
TAPE_2007_EXAMPLE = os.path.join(SPECFEM2D_ORIGINAL, "EXAMPLES", "Tape2007")

# The SPECFEM2D working directory that we will create separate from the downloaded repo
SPECFEM2D_WORKDIR = os.path.join(WORKDIR, "specfem2d_workdir")
SPECFEM2D_BIN = os.path.join(SPECFEM2D_WORKDIR, "bin")
SPECFEM2D_DATA = os.path.join(SPECFEM2D_WORKDIR, "DATA")
SPECFEM2D_OUTPUT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES")

# Pre-defined locations of velocity models we will generate using the solver
SPECFEM2D_MODEL_INIT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_INIT")
SPECFEM2D_MODEL_TRUE = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_TRUE")

In [3]:
# Incase we've run this docs page before, delete the working directory before remaking
if os.path.exists(SPECFEM2D_WORKDIR):
    shutil.rmtree(SPECFEM2D_WORKDIR)

os.mkdir(SPECFEM2D_WORKDIR)
os.chdir(SPECFEM2D_WORKDIR)

# Copy the binary files incase we update the source code. These can also be symlinked.
shutil.copytree(SPECFEM2D_BIN_ORIGINAL, "bin")

# Copy the DATA/ directory because we will be making edits here frequently and it's useful to
# retain the original files for reference. We will be running one of the example problems: Tape2007
shutil.copytree(os.path.join(TAPE_2007_EXAMPLE, "DATA"), "DATA")
!pwd
!ls

/home/scoped/work/specfem2d_workdir
bin  DATA


In [4]:
# Run the Tape2007 example to make sure SPECFEM2D is working as expected
os.chdir(TAPE_2007_EXAMPLE)
!./run_this_example.sh > output_log.txt

assert(os.path.exists("OUTPUT_FILES/forward_image000004800.jpg")), \
    (f"Example did not run, the remainder of this docs page will likely not work."
     f"Please check the following directory: {TAPE_2007_EXAMPLE}")

!tail output_log.txt

 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------
 D a t e : 01 - 03 - 2023                                 T i m e  : 03:09:37
 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------

see results in directory: OUTPUT_FILES/

done
Wed 01 Mar 2023 03:09:37 AM UTC


In [5]:
# First we will set the correct SOURCE and STATION files.
# This is the same task as shown in ./run_this_example.sh
os.chdir(SPECFEM2D_DATA)

# Symlink source 001 as our main source
if os.path.exists("SOURCE"):
    os.remove("SOURCE")
os.symlink("SOURCE_001", "SOURCE")

# Copy the correct Par_file so that edits do not affect the original file
if os.path.exists("Par_file"):
    os.remove("Par_file")
shutil.copy("Par_file_Tape2007_onerec", "Par_file")

!ls

interfaces_Tape2007.dat		     SOURCE_003  SOURCE_012  SOURCE_021
model_velocity.dat_checker	     SOURCE_004  SOURCE_013  SOURCE_022
Par_file			     SOURCE_005  SOURCE_014  SOURCE_023
Par_file_Tape2007_132rec_checker     SOURCE_006  SOURCE_015  SOURCE_024
Par_file_Tape2007_onerec	     SOURCE_007  SOURCE_016  SOURCE_025
proc000000_model_velocity.dat_input  SOURCE_008  SOURCE_017  STATIONS
SOURCE				     SOURCE_009  SOURCE_018  STATIONS_checker
SOURCE_001			     SOURCE_010  SOURCE_019
SOURCE_002			     SOURCE_011  SOURCE_020


### Generate initial model

In [6]:
os.chdir(SPECFEM2D_DATA)

# Ensure that SPECFEM2D outputs the velocity model in the expected binary format
!seisflows sempar NPROC 1  # allow creation of .bin files
!seisflows sempar setup_with_binary_database 1  # allow creation of .bin files
!seisflows sempar save_model binary  # output model in .bin database format
!seisflows sempar save_ascii_kernels .false.  # output kernels in .bin format, not ASCII

NPROC: 1 -> 1
setup_with_binary_database: 0 -> 1
SAVE_MODEL: default -> binary
save_ASCII_kernels: .true. -> .false.


In [7]:
# SPECFEM requires that we create the OUTPUT_FILES directory before running
os.chdir(SPECFEM2D_WORKDIR)

if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)

os.mkdir(SPECFEM2D_OUTPUT)

!ls

bin  DATA  OUTPUT_FILES


In [8]:
# GENERATE MODEL_INIT
os.chdir(SPECFEM2D_WORKDIR)
!pwd
# Run the mesher and solver to generate our initial model
!./bin/xmeshfem2D > OUTPUT_FILES/mesher_log.txt
!./bin/xspecfem2D > OUTPUT_FILES/solver_log.txt

# Move all the relevant files into OUTPUT_FILES
!mv ./DATA/*bin OUTPUT_FILES
!mv OUTPUT_FILES OUTPUT_FILES_INIT

!head OUTPUT_FILES_INIT/solver_log.txt
!tail OUTPUT_FILES_INIT/solver_log.txt

/home/scoped/work/specfem2d_workdir

 **********************************************
 **** Specfem 2-D Solver - MPI version     ****
 **********************************************

 Running Git version of the code corresponding to commit 604f766101c9ece3d588d61e6275d81d50dd8e65
 dating From Date:   Thu Sep 22 12:55:08 2022 +0200

 There are            1  MPI processes
 Processes are numbered from 0 to            0
 -------------------------------------------------------------------------------
 Program SPECFEM2D: 
 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------
 Tape-Liu-Tromp (GJI 2007)
 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------
 D a t e : 01 - 03 - 2023                                 T i m e  : 03:09:49
 -------------------------------------------------

### Generate true model

In [9]:
# GENERATE MODEL_TRUE
os.chdir(SPECFEM2D_WORKDIR)
shutil.rmtree("DATA")

# Copy the DATA/ directory because we will be making edits here frequently and it's useful to
# retain the original files for reference. We will be running one of the example problems: Tape2007
shutil.copytree(os.path.join(TAPE_2007_EXAMPLE, "DATA"), "DATA")
!pwd
!ls

os.chdir(SPECFEM2D_DATA)
# Symlink source 001 as our main source
if os.path.exists("SOURCE"):
    os.remove("SOURCE")
os.symlink("SOURCE_001", "SOURCE")

# Copy the correct Par_file so that edits do not affect the original file
if os.path.exists("Par_file"):
    os.remove("Par_file")
shutil.copy("Par_file_Tape2007_onerec", "Par_file")
!ls

# Edit the Par_file to use checkboard data from Tape 2007
os.chdir(SPECFEM2D_DATA)
!seisflows sempar NPROC 1 
!seisflows sempar model legacy 
!seisflows sempar setup_with_binary_database 1  # allow creation of .bin files
!seisflows sempar save_model binary  # output model in .bin database format
!seisflows sempar save_ascii_kernels .false.  # output kernels in .bin format, not ASCII

/home/scoped/work/specfem2d_workdir
bin  DATA  OUTPUT_FILES_INIT
interfaces_Tape2007.dat		     SOURCE_003  SOURCE_012  SOURCE_021
model_velocity.dat_checker	     SOURCE_004  SOURCE_013  SOURCE_022
Par_file			     SOURCE_005  SOURCE_014  SOURCE_023
Par_file_Tape2007_132rec_checker     SOURCE_006  SOURCE_015  SOURCE_024
Par_file_Tape2007_onerec	     SOURCE_007  SOURCE_016  SOURCE_025
proc000000_model_velocity.dat_input  SOURCE_008  SOURCE_017  STATIONS
SOURCE				     SOURCE_009  SOURCE_018  STATIONS_checker
SOURCE_001			     SOURCE_010  SOURCE_019
SOURCE_002			     SOURCE_011  SOURCE_020
NPROC: 1 -> 1
MODEL: default -> legacy
setup_with_binary_database: 0 -> 1
SAVE_MODEL: default -> binary
save_ASCII_kernels: .true. -> .false.


In [10]:
# Re-run the mesher and solver to generate our target velocity model
os.chdir(SPECFEM2D_WORKDIR)

# Make sure the ./OUTPUT_FILES directory exists since we moved the old one
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)

# Run the binaries to generate MODEL_TRUE
!./bin/xmeshfem2D > OUTPUT_FILES/mesher_log.txt
!./bin/xspecfem2D > OUTPUT_FILES/solver_log.txt

# Move all the relevant files into OUTPUT_FILES
!mv ./DATA/*bin OUTPUT_FILES
!mv OUTPUT_FILES OUTPUT_FILES_TRUE

!head OUTPUT_FILES_TRUE/solver_log.txt
!tail OUTPUT_FILES_TRUE/solver_log.txt


 **********************************************
 **** Specfem 2-D Solver - MPI version     ****
 **********************************************

 Running Git version of the code corresponding to commit 604f766101c9ece3d588d61e6275d81d50dd8e65
 dating From Date:   Thu Sep 22 12:55:08 2022 +0200

 There are            1  MPI processes
 Processes are numbered from 0 to            0
 -------------------------------------------------------------------------------
 Program SPECFEM2D: 
 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------
 Tape-Liu-Tromp (GJI 2007)
 -------------------------------------------------------------------------------
 -------------------------------------------------------------------------------
 D a t e : 01 - 03 - 2023                                 T i m e  : 03:10:04
 -------------------------------------------------------------------------------
 ----

In [11]:
# Great, we have all the necessary SPECFEM files to run our SeisFlows inversion!
!ls

bin  DATA  OUTPUT_FILES_INIT  OUTPUT_FILES_TRUE


In [12]:
# Copy stations file to run inversion with the 132 stations of the checker example
os.chdir(SPECFEM2D_DATA)
shutil.copy("STATIONS", "STATIONS_one")
shutil.copy("STATIONS_checker", "STATIONS")
!head STATIONS

S000000 AA 2.43610e+05 2.78904e+05 0.0 0.0
S000001 AA 3.38981e+05 1.77849e+05 0.0 0.0
S000002 AA 1.64438e+05 2.94733e+05 0.0 0.0
S000003 AA 9.22250e+04 3.68887e+05 0.0 0.0
S000004 AA 2.90702e+05 2.46865e+05 0.0 0.0
S000005 AA 2.85186e+05 2.09065e+05 0.0 0.0
S000006 AA 4.26783e+05 1.80623e+05 0.0 0.0
S000007 AA 2.59207e+05 1.70701e+05 0.0 0.0
S000008 AA 3.75965e+05 2.18298e+05 0.0 0.0
S000009 AA 2.22832e+05 2.44253e+05 0.0 0.0
